In [1]:
import seaborn as sns
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from IPython.display import display, HTML

#import my modules
import acquire as a
import prepare as p

from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Acquire

In [2]:
df = a.get_zillow()

In [3]:
df.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,regionidcounty,regionidzip,typeconstructiondesc,unitcnt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,NaN,3.5,4.0,NaN,3100.0,6059.0,NaN,2.0,NaN,...,1286.0,96978.0,NaN,NaN,485713.0,537569.0,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,NaN,1.0,2.0,NaN,1465.0,6111.0,1.0,1.0,NaN,...,2061.0,97099.0,NaN,NaN,88000.0,376000.0,464000.0,5672.48,0.055619,2017-01-01
2,14186244,NaN,2.0,3.0,NaN,1243.0,6059.0,NaN,2.0,NaN,...,1286.0,97078.0,NaN,NaN,85289.0,479489.0,564778.0,6488.30,0.005383,2017-01-01
3,12177905,NaN,3.0,4.0,NaN,2376.0,6037.0,NaN,NaN,NaN,...,3101.0,96330.0,NaN,1.0,108918.0,36225.0,145143.0,1777.51,-0.103410,2017-01-01
4,10887214,NaN,3.0,3.0,NaN,1312.0,6037.0,NaN,NaN,NaN,...,3101.0,96451.0,NaN,1.0,73681.0,45726.0,119407.0,1533.89,0.006940,2017-01-01


In [4]:
df.shape

(77579, 36)

In [5]:
#getting the latest transactions 
df = df.sort_values(by ='transactiondate', ascending=True).drop_duplicates( subset = 'parcelid' ,keep= 'last')
  

In [6]:
df.shape

(77381, 36)

In [7]:
#this list has all types of single unit properties
single= ['Single Family Residential',' Mobile Home' , 'Townhouse ', 'Manufactured, Modular, Prefabricated Homes'  ]

In [8]:
#create a mask
single_mask = df['propertylandusedesc'].isin(single)

In [9]:
#if unitcnt == that's means is is a single property but if it is null we can compare with our list of single

In [27]:
#using that mask and also add  a condition
df_single = df[single_mask & ((df['unitcnt'] == 1) | (df['unitcnt'].isnull()))]
    

In [28]:
#this df contains all single unit homes of 2017 with the latest transaction
df_single.shape

(52350, 36)

In [29]:
a.summarize(df_single)

Dataframe shape: 
(52350, 36)
Dataframe head: 
   parcelid  basementsqft  bathroomcnt  bedroomcnt  decktypeid  \
0  14297519           NaN          3.5         4.0         NaN   
1  17052889           NaN          1.0         2.0         NaN   
2  14186244           NaN          2.0         3.0         NaN   

   calculatedfinishedsquarefeet    fips  fireplacecnt  garagecarcnt  \
0                        3100.0  6059.0           NaN           2.0   
1                        1465.0  6111.0           1.0           1.0   
2                        1243.0  6059.0           NaN           2.0   

   hashottuborspa  ...  regionidcounty  regionidzip  typeconstructiondesc  \
0             NaN  ...          1286.0      96978.0                   NaN   
1             NaN  ...          2061.0      97099.0                   NaN   
2             NaN  ...          1286.0      97078.0                   NaN   

   unitcnt  structuretaxvaluedollarcnt  landtaxvaluedollarcnt  \
0      NaN                   

Central      11868
Yes           1567
None           159
Wall Unit       16
Name: airconditioningdesc, dtype: int64
**architecturalstyletypeid**
Unique Values :  5
 
2.0      2
3.0      3
7.0     62
8.0      1
21.0     2
Name: architecturalstyletypeid, dtype: int64
**architecturalstyledesc**
Unique Values :  5
 
Contemporary     62
Cape Cod          3
Bungalow          2
Ranch/Rambler     2
Conventional      1
Name: architecturalstyledesc, dtype: int64
**storytypeid**
Unique Values :  1
 
7.0    47
Name: storytypeid, dtype: int64
**storydesc**
Unique Values :  1
 
Basement    47
Name: storydesc, dtype: int64
**propertylandusetypeid**
Unique Values :  2
 
261.0    52292
275.0       58
Name: propertylandusetypeid, dtype: int64
**propertylandusedesc**
Unique Values :  2
 
Single Family Residential                     52292
Manufactured, Modular, Prefabricated Homes       58
Name: propertylandusedesc, dtype: int64
**typeconstructiontypeid**
Unique Values :  2
 
4.0     1
6.0    75
Name: ty

**takeaways**
- numberofstories there is one property with 6 stories , I would drop it!
- unitcnt is not needed any more. I would drop it
- there are a lot of nulls so in preparation step I will handle them

# Preparation

In [30]:
p.miss_dup_values(df_single)

Your selected dataframe has 36 columns.
There are 25 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
basementsqft,52303,99.9
storydesc,52303,99.9
storytypeid,52303,99.9
architecturalstyledesc,52280,99.9
architecturalstyletypeid,52280,99.9
typeconstructiontypeid,52274,99.9
typeconstructiondesc,52274,99.9
decktypeid,51960,99.3
hashottuborspa,50836,97.1
fireplacecnt,45120,86.2


In [31]:
df_single.shape

(52350, 36)

In [32]:
# I will drop columns and rows with a lot of null values 
#here I will include unitcnt because the value is 1 for all the properties . as it is the next to 65.6% 
#I can include it in my function
df_single= p.handle_missing_values(df_single, prop_required_columns=0.75, prop_required_row=0.75)

In [33]:
df_single.shape

(52348, 20)

In [34]:
p.miss_dup_values(df_single)

Your selected dataframe has 20 columns.
There are 9 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
regionidcity,1034,2.0
lotsizesquarefeet,364,0.7
yearbuilt,118,0.2
structuretaxvaluedollarcnt,87,0.2
calculatedfinishedsquarefeet,86,0.2
regionidzip,25,0.0
taxamount,4,0.0
landtaxvaluedollarcnt,1,0.0
taxvaluedollarcnt,1,0.0


In [37]:

#all these  columns represent a minimun amount so I can drop them 
df_single= p.drop_low_missing_values(df_single, per= 3 )

In [38]:
df_single.shape

(50819, 20)